#### Goal
Get images linked back to their titles

In [1]:
import sys
import os
import pandas as pd
import boto3

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

sys.path.append("..")

In [2]:
import keys
import data_fcns as dfc

Images directory:

I set up an s3 bucket: https://comrx.s3-us-west-2.amazonaws.com/covers/

In [3]:
# Define path to secret
secret_path_aws = os.path.join(os.environ['HOME'], '.secret', 
                           'aws_ps_flatiron.json')
secret_path_aws

aws_keys = keys.get_keys(secret_path_aws)
user = aws_keys['user']
ps = aws_keys['password']
host = aws_keys['host']
db = aws_keys['db_name']

aws_ps_engine = ('postgresql://' + user + ':' + ps + '@' + host + '/' + db)

# Setup PSQL connection
conn = psql.connect(
    database=db,
    user=user,
    password=ps,
    host=host,
    port='5432'
)

In [8]:
# Instantiate cursor
cur = conn.cursor()

In [53]:
#  Count records.
query = """
    SELECT * from comics LIMIT 100;
"""

In [50]:
#  Count records.
query_alter = """
    ALTER TABLE comics 
    ADD COLUMN img_url VARCHAR;
"""

In [30]:
tables = """
SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='public'
   AND table_type='BASE TABLE';
   """

ALTER TABLE table_name
ADD COLUMN new_column_name data_type;

In [51]:
# Execute the query
cur.execute(query_alter)

In [54]:
# Execute the query
cur.execute(query)

In [55]:
conn.commit()

In [56]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [69]:
temp_df.head()

,comic_id,comic_title,img_url
0,1,0Secret Wars (Marvel),None
1,2,100 Bullets Brother Lono (DC),None
2,3,100 Penny Press Locke & Key (IDW),None
3,4,100 Penny Press Star Trek (IDW),None
4,5,100 Penny Press Thunder Agent (IDW),None


Cool. Let's practice updating.

In [74]:
query_update = """
UPDATE comics SET img_url= 'https://comrx.s3-us-west-2.amazonaws.com/covers/wonder_woman.jpg'
WHERE comic_title = 'Wonder Woman (DC)';
"""

In [80]:
#  Count records.
query = """
    SELECT * from comics WHERE comic_title = 'Wonder Woman (DC)';
"""

In [81]:
# Execute the query
cur.execute(query)

In [77]:
# Execute the query
cur.execute(query_update)

In [76]:
conn.rollback()

In [78]:
conn.commit()

In [82]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [83]:
temp_df.head()

,comic_id,comic_title,img_url
0,7021,Wonder Woman (DC),https://comrx.s3-us-west-2.amazonaws.com/cover...


Yay.

In [58]:
!ls

__init__.py
archive
assets
comic_recs_part.py
comics_rx-1_data_prep.ipynb
comics_rx-2_eda.ipynb
comics_rx-3_als_all_data.ipynb
comics_rx-4_als_reduced_data.ipynb
comics_rx-5_pseudo_deploy.ipynb
comics_rx-6_als_reduced_data_gs_cv.ipynb
comics_rx-7_mvp_dev.ipynb
comics_rx-8_recommender_poc.ipynb
comics_rx-9_scrape_comic_covers.ipynb
comics_rx-9_scrape_comic_covers_win.ipynb
drivers
get_images.ipynb
models
raw_data
references
scratch
support_data
versions


We need a crosswalk from `comic_title` to `img_url`. For that we need to rebuild the `title` I've been using to search on images, because that's what I've been using to label the jpegs.

In [31]:
#  Count records.
query = """
    SELECT * from comic_trans;
"""

In [32]:
# Execute the query
cur.execute(query)

In [33]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [34]:
temp_df.head(3)

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG)
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG)
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG)


In [35]:
temp_df['title'] = ( temp_df['title_and_num'].apply(dfc.cut_issue_num) )

In [36]:
temp_df.head()

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title,title
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG),Filler Bunny
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG),Gargoyles
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG),Royal Historian of Oz
3,5,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-14 19:49:40,00399,Royal Historian of Oz (SLG),Royal Historian of Oz
4,6,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-19 10:39:04,00237,Royal Historian of Oz (SLG),Royal Historian of Oz


In [37]:
temp_df['title'] = (temp_df['title'].apply(lambda x : x.replace('&' ,'and'))
                    .apply(lambda x : x.replace('?' ,''))
                    .apply(lambda x : x.replace('/' ,' '))
                   )

In [38]:
temp_df.head()

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title,title
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG),Filler Bunny
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG),Gargoyles
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG),Royal Historian of Oz
3,5,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-14 19:49:40,00399,Royal Historian of Oz (SLG),Royal Historian of Oz
4,6,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-19 10:39:04,00237,Royal Historian of Oz (SLG),Royal Historian of Oz


In [39]:
work = temp_df.groupby(['comic_title', 'title'], as_index=False).index.count()

In [40]:
work.head()

,comic_title,title,index
0,0Secret Wars (Marvel),0Secret Wars,1
1,1 For $1 Abe Sapien (Dark Horse),1 For $1 Abe Sapien,3
2,1 For $1 Action Philosopher (Dark Horse),1 For $1 Action Philosopher,3
3,1 For $1 Avatar Last Airbende (Dark Horse),1 For $1 Avatar Last Airbende,2
4,1 For $1 Axe Cop Bad Guy Eart (Dark Horse),1 For $1 Axe Cop Bad Guy Eart,1


In [41]:
work.shape

(7202, 3)

In [42]:
work['filename'] = work['title'].apply(lambda x: x.lower().replace(' ', '_')) + '.jpg'

In [44]:
search_path = 'https://comrx.s3-us-west-2.amazonaws.com/covers/'
search_path

'https://comrx.s3-us-west-2.amazonaws.com/covers/'

In [45]:
work['search_path'] = search_path + work['filename']

In [153]:
work.loc[work['title']=='Wonder Woman']['search_path']

7011    https://comrx.s3-us-west-2.amazonaws.com/cover...
7012    https://comrx.s3-us-west-2.amazonaws.com/cover...
Name: search_path, dtype: object

In [162]:
work.loc[work['comic_title']=='Wonder Woman (DC)']['filename'].values[0]

'wonder_woman.jpg'

In [46]:
work.head()

,comic_title,title,index,filename,search_path
0,0Secret Wars (Marvel),0Secret Wars,1,0secret_wars.jpg,https://comrx.s3-us-west-2.amazonaws.com/cover...
1,1 For $1 Abe Sapien (Dark Horse),1 For $1 Abe Sapien,3,1_for_$1_abe_sapien.jpg,https://comrx.s3-us-west-2.amazonaws.com/cover...
2,1 For $1 Action Philosopher (Dark Horse),1 For $1 Action Philosopher,3,1_for_$1_action_philosopher.jpg,https://comrx.s3-us-west-2.amazonaws.com/cover...
3,1 For $1 Avatar Last Airbende (Dark Horse),1 For $1 Avatar Last Airbende,2,1_for_$1_avatar_last_airbende.jpg,https://comrx.s3-us-west-2.amazonaws.com/cover...
4,1 For $1 Axe Cop Bad Guy Eart (Dark Horse),1 For $1 Axe Cop Bad Guy Eart,1,1_for_$1_axe_cop_bad_guy_eart.jpg,https://comrx.s3-us-west-2.amazonaws.com/cover...


In [202]:
tests = ['Wonder Woman (DC)', 'Batman (DC)', 'X-Men (Marvel)', 'Werlindo']

In [47]:
client = boto3.client('s3')

In [178]:
ttl = 'wonder woman (dc)'
url = 'www.overhere.com'

In [188]:
ssql = ("SELECT * FROM '" + ttl + "';" + 
       "WHERE huh ")

In [189]:
ssql

"SELECT * FROM 'wonder woman (dc)';WHERE huh "

In [240]:
for test in tests:
    # Find file name
    try:
        got_filename = work.loc[work['comic_title']==test]['filename'].values[0]
        client.head_object(Bucket='comrx', Key='covers/' + got_filename)
        print('got ' + got_filename)
        
        # Update comics table on AWS
        img_url = work.loc[work['comic_title']==test]['search_path'].values[0]
        print(img_url)
        
        # Build query string
        query_update = ("UPDATE comics " + 
                        "SET img_url = '" + img_url + "' " +
                        "WHERE comic_title = '" + test +
                        "';"
                       )
        print(query_update)
        
        # Execute query
        cur.execute(query_update)
        conn.commit()
           
    except:
        print('No Match')


got wonder_woman.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/wonder_woman.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/wonder_woman.jpg' WHERE comic_title = 'Wonder Woman (DC)';
got batman.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/batman.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/batman.jpg' WHERE comic_title = 'Batman (DC)';
got x-men.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/x-men.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/x-men.jpg' WHERE comic_title = 'X-Men (Marvel)';
No Match


In [194]:
#  Count records.
query = """
    SELECT * from comics WHERE comic_title = 'Wonder Woman (DC)';
"""

In [215]:
#  Count records.
query = """
    SELECT * from comics WHERE comic_title = 'Batman (DC)';
"""

In [223]:
#  Count records.
query = """
    SELECT * from comics WHERE comic_title = 'X-Men (Marvel)';
"""

In [224]:
# Execute the query
cur.execute(query)

In [225]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [226]:
temp_df.head()

,comic_id,comic_title,img_url
0,7126,X-Men (Marvel),https://comrx.s3-us-west-2.amazonaws.com/cover...


Great! Let's make a full list of all comics titles.

In [257]:
title_search = list(work['comic_title'])

In [258]:
len(title_search)

7202

In [245]:
title_search[:5]

['0Secret Wars (Marvel)',
 '1 For $1 Abe Sapien (Dark Horse)',
 '1 For $1 Action Philosopher (Dark Horse)',
 '1 For $1 Avatar Last Airbende (Dark Horse)',
 '1 For $1 Axe Cop Bad Guy Eart (Dark Horse)']

In [56]:
generic_img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg'

In [247]:
for title_search in title_search[:2]:
    # Find file name
    try:
        got_filename = work.loc[work['comic_title']==title_search]['filename'].values[0]
        client.head_object(Bucket='comrx', Key='covers/' + got_filename)
        print('got ' + got_filename)
        
        # Update comics table on AWS
        img_url = work.loc[work['comic_title']==title_search]['search_path'].values[0]
        print(img_url)
        
        # Build query string
        query_update = ("UPDATE comics " + 
                        "SET img_url = '" + img_url + "' " +
                        "WHERE comic_title = '" + title_search +
                        "';"
                        )
        print(query_update)
        
        # Execute query
        cur.execute(query_update)
        conn.commit()
           
    except:
        print('No Match, use generic image')
                # Build query string
        query_update = ("UPDATE comics " + 
                        "SET img_url = '" + generic_img_url + "' " +
                        "WHERE comic_title = '" + title_search +
                        "';"
                        )
        print(query_update)
        
        # Execute query
        cur.execute(query_update)
        conn.commit()


No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = '0Secret Wars (Marvel)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = '1 For $1 Abe Sapien (Dark Horse)';


Let's check!

---

In [248]:
check_sql = "SELECT * FROM comics WHERE comic_title = '0Secret Wars (Marvel)';"

In [249]:
cur.execute(check_sql)

In [250]:
# Check results
check_df = pd.DataFrame(cur.fetchall())
check_df.columns = [col.name for col in cur.description]

In [19]:
pd.set_option('display.max_colwidth', -1)

In [253]:
check_df.head()

,comic_id,comic_title,img_url
0,1,0Secret Wars (Marvel),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg


Looks good. Let's do it!

In [259]:
for title in title_search:
    print(title)

0Secret Wars (Marvel)
1 For $1 Abe Sapien (Dark Horse)
1 For $1 Action Philosopher (Dark Horse)
1 For $1 Avatar Last Airbende (Dark Horse)
1 For $1 Axe Cop Bad Guy Eart (Dark Horse)
1 For $1 Captain Midnight (Dark Horse)
1 For $1 Conan the Barbarian (Dark Horse)
1 For $1 Dragon Age Silent Gr (Dark Horse)
1 For $1 Mass Effect Foundati (Dark Horse)
1 For $1 Star Wars Legacy (Dark Horse)
1 For $1 Strain (Dark Horse)
1 For $1 Usagi Yojimb (Dark Horse)
1 For $1 X (Dark Horse)
1 For $1 the Victories (Dark Horse)
1 For 1 Baltimore Plague Ship (Dark Horse)
1 For 1 Star Wars (C: 1-0-0 (Dark Horse)
100 Bullets Brother Lono (DC)
100 Penny Press Locke & Key (IDW)
100 Penny Press Star Trek (IDW)
100 Penny Press Thunder Agent (IDW)
100 Penny Press Tmnt (IDW)
100th Anniversary Special (Marvel)
12 Reasons To Die (Other)
13 Coins (Other)
13th Artifact One Sho (Topcow)
1602 Witch Hunter Angela (Marvel)
1872 (Marvel)
1985 Black Hole Repo (Other)
2021 Lost Children (Other)
21st Century Tank Girl (Other)
2

Green Wake (Image)
Grendel Vs Shadow (Dark Horse)
Grievous Journey of Ichabod A (Other)
Grifter (DC)
Grifter (Other)
Grim Leaper (Image)
Grimm (DE)
Grimm Fairy Tales (Zenescope)
Grimm Fairy Tales 2017 Holida (Zenescope)
Grimm Fairy Tales 2018 Holida (Zenescope)
Grimm Fairy Tales 2019 Annua (Zenescope)
Grimm Fairy Tales Annual 201 (Zenescope)
Grimm Fairy Tales Annual Las (Zenescope)
Grimm V2 (DE)
Grimm Warlock (DE)
Grindhouse Doors Open At Midn (Dark Horse)
Grindhouse Drive In Bleed Ou (Dark Horse)
Grizzly Shark (Image)
Groo Fray of the Gods (Dark Horse)
Groo Friends and Foes (Dark Horse)
Groo Hogs of Horder (Dark Horse)
Groo Play of Gods (Dark Horse)
Groo Vs Conan (Dark Horse)
Groot (Marvel)
Grrl Scouts Magic Socks (Image)
Grumble (Other)
Grumpy Cat & Pokey (DE)
Grumpy Cat (DE)
Grumpy Cat Garfield (DE)
Guardians 3000 (Marvel)
Guardians Team-Up (Marvel)
Guardians of Galaxy (Marvel)
Guardians of Galaxy Annual (Marvel)
Guardians of Galaxy Best Stor (Marvel)
Guardians of Galaxy Galaxys M (

Skyman (Dark Horse)
Skyward (Image)
Skyward (Other)
Slam (Boom)
Slam Next Jam (Boom)
Slapstick (Marvel)
Slash & Burn (Vertigo)
Slasher (Other)
Slayer Repentless (Dark Horse)
Sledgehammer 44 (Dark Horse)
Sledgehammer 44 Lightning Wa (Dark Horse)
Sleeper Ca (Other)
Sleepless (Image)
Sleepy Hollow (Boom)
Sleepy Hollow Origins (Boom)
Sleepy Hollow Providence (Boom)
Sleigher Heavy Metal Santa Cl (Other)
Slots #1 Cvr B Walking Dead (Image)
Slots (Image)
Smallville Alien (DC)
Smallville Season 11 (DC)
Smallville Season 11 (Other)
Smallville Season 11 Alien (DC)
Smallville Season 11 Chaos (DC)
Smallville Season 11 Continui (DC)
Smallville Season 11 Lantern (DC)
Smallville Season 11 Special (DC)
Smoke and Mirrors (IDW)
Smooth Criminals (Boom)
Snake Eyes & Storm Shadow (IDW)
Snake Eyes (Idw) (IDW)
Snake Eyes Ongoing (Idw) (IDW)
Snap Flash Hustle (Other)
Snapshot (Image)
Snotgirl (Image)
Snow Blind (Boom)
Snowfall (Image)
Solar Flare (Other)
Solar Flare Season 2 (Other)
Solar Flare Season Three (

In [275]:
for title in title_search[1049:]:
    # Find file name
    
    # escape single count
    title_sql = title.replace("'", "''")
    
    try:
        got_filename = work.loc[work['comic_title']==title]['filename'].values[0]
        client.head_object(Bucket='comrx', Key='covers/' + got_filename)
        print('got ' + got_filename)
        
        # Update comics table on AWS
        img_url = work.loc[work['comic_title']==title]['search_path'].values[0]
        print(img_url)
        
        # Build query string
        query_update = ("UPDATE comics " + 
                        "SET img_url = '" + img_url + "' " +
                        "WHERE comic_title = '" + title_sql +
                        "';"
                        )
        print(query_update)
        
        # Execute query
        cur.execute(query_update)
        conn.commit()
           
    except:
        print('No Match, use generic image')
                # Build query string
        query_update = ("UPDATE comics " + 
                        "SET img_url = '" + generic_img_url + "' " +
                        "WHERE comic_title = '" + title_sql +
                        "';"
                        )
        print(query_update)
        
        # Execute query
        cur.execute(query_update)
        conn.commit()


No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Cap''n Dinosaur One Sho (Image)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Cape (One Shot) Legacy E (IDW)';
got cape_1969.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/cape_1969.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/cape_1969.jpg' WHERE comic_title = 'Cape 1969 (IDW)';
got capt_action_cat.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/capt_action_cat.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/capt_action_cat.jpg' WHERE comic_title = 'Capt Action Cat (DE)';
got capt_victory_and_galactic_range.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/capt_victory_and_galactic_range.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/cap

got captain_america_and_namor.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/captain_america_and_namor.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/captain_america_and_namor.jpg' WHERE comic_title = 'Captain America and Namor (Marvel)';
got captain_america_and_secret_av.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/captain_america_and_secret_av.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/captain_america_and_secret_av.jpg' WHERE comic_title = 'Captain America and Secret Av (Marvel)';
got captain_atom.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/captain_atom.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/captain_atom.jpg' WHERE comic_title = 'Captain Atom (DC)';
got captain_atom.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/captain_atom.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/captain_atom.jpg' WHERE comic_title = 'Capt

got cataclysm.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/cataclysm.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/cataclysm.jpg' WHERE comic_title = 'Cataclysm (Marvel)';
got cataclysm_ultimate_spider-ma.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/cataclysm_ultimate_spider-ma.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/cataclysm_ultimate_spider-ma.jpg' WHERE comic_title = 'Cataclysm Ultimate Spider-Ma (Marvel)';
got cataclysm_ultimate_x-men.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/cataclysm_ultimate_x-men.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/cataclysm_ultimate_x-men.jpg' WHERE comic_title = 'Cataclysm Ultimate X-Men (Marvel)';
got cataclysm_ultimates.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/cataclysm_ultimates.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/cataclysm_ultimates.jpg' WHERE comic_title = 

got cavewoman_natural_selection.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/cavewoman_natural_selection.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/cavewoman_natural_selection.jpg' WHERE comic_title = 'Cavewoman Natural Selection (Other)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Cavewoman Night Out One-Sho (Other)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Cavewoman Night Out One-Shot (Other)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Cavewoman Oasis (Other)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Cavewoman Prehistoric Pinups (Other)';
got c

got chasing_the_dead.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/chasing_the_dead.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/chasing_the_dead.jpg' WHERE comic_title = 'Chasing the Dead (IDW)';
got chastity.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/chastity.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/chastity.jpg' WHERE comic_title = 'Chastity (DE)';
got chew_#13_2nd_ptg_(mr)_(pp.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/chew_#13_2nd_ptg_(mr)_(pp.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/chew_#13_2nd_ptg_(mr)_(pp.jpg' WHERE comic_title = 'Chew #13 2nd Ptg (Mr) (Pp (Image)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Chew #2 4th Ptg (Mr) (Pp (Image)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/cov

...

No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'World of Archie Jumbo Comics (Archie)';
got world_of_tanks.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/world_of_tanks.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/world_of_tanks.jpg' WHERE comic_title = 'World of Tanks (Dark Horse)';
got world_of_tanks_citadel.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/world_of_tanks_citadel.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/world_of_tanks_citadel.jpg' WHERE comic_title = 'World of Tanks Citadel (Dark Horse)';
got worlds_finest.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/worlds_finest.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/worlds_finest.jpg' WHERE comic_title = 'Worlds Finest (DC)';
got worlds_finest.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/worlds_finest.

No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'X-23 Mgc (Marvel)';
got x-club.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/x-club.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/x-club.jpg' WHERE comic_title = 'X-Club (Marvel)';
got x-factor.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/x-factor.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/x-factor.jpg' WHERE comic_title = 'X-Factor (Marvel)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'X-Files (2016) (IDW)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'X-Files Annual 201 (IDW)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaw

got x-men_giant-size.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/x-men_giant-size.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/x-men_giant-size.jpg' WHERE comic_title = 'X-Men Giant-Size (Marvel)';
got x-men_gold.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/x-men_gold.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/x-men_gold.jpg' WHERE comic_title = 'X-Men Gold (Marvel)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'X-Men Gold Annual (Marvel)';
got x-men_grand_design.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/x-men_grand_design.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/x-men_grand_design.jpg' WHERE comic_title = 'X-Men Grand Design (Marvel)';
got x-men_grand_design_second_gen.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/x-men_grand_design_second_gen.jpg

No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Yankee (Other)';
got year_of_marvels_incredible.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/year_of_marvels_incredible.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/year_of_marvels_incredible.jpg' WHERE comic_title = 'Year of Marvels Incredible (Marvel)';
got years_of_future_past.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/years_of_future_past.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/years_of_future_past.jpg' WHERE comic_title = 'Years of Future Past (Marvel)';
got yi_soon_shin.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/yi_soon_shin.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/yi_soon_shin.jpg' WHERE comic_title = 'Yi Soon Shin (Other)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us

No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Zombie Tramp Xxxmas Special R (Other)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Zombie Tramp Xxxmas Special T (Other)';
got zombie_war.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/zombie_war.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/zombie_war.jpg' WHERE comic_title = 'Zombie War (IDW)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Zombies Assembl (Marvel)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'Zombies Assemble (Marvel)';
got zombies_assemble_2.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/zombies

In [274]:
conn.rollback()

In [ ]:
Cap'n Dinosaur One Sho (Image)

In [263]:
work.loc[work['comic_title']=="Cap'n Dinosaur One Sho (Image)",]

,comic_title,title,index,search_path,filename
1049,Cap'n Dinosaur One Sho (Image),Cap'n Dinosaur One Sho,34,https://comrx.s3-us-west-2.amazonaws.com/covers/cap'n_dinosaur_one_sho.jpg,cap'n_dinosaur_one_sho.jpg


In [264]:
y = "Cap'n Dinosaur One Sho (Image)"

In [265]:
y

"Cap'n Dinosaur One Sho (Image)"

In [266]:
y.replace("'", "''")

"Cap''n Dinosaur One Sho (Image)"

In [268]:
i = 0

for t in title_search:
    if t != y:
        i+=1
    else:
        break
i

1049

In [276]:
title_search[1040:1050]

['Caligula (Avatar)',
 'Caligula Heart of Rome (Avatar)',
 'Call of Duty Black Ops Iii (Dark Horse)',
 'Call of Duty Zombies (Dark Horse)',
 'Call of Duty Zombies 2 (Dark Horse)',
 'Calling Cthulhu Chronicles (Boom)',
 'Calvario Hills (Fantagraphics)',
 'Canadian Vark (Other)',
 'Cannibal (Image)',
 "Cap'n Dinosaur One Sho (Image)"]

In [272]:
title_search[1049:]

["Cap'n Dinosaur One Sho (Image)",
 'Cape (One Shot) Legacy E (IDW)',
 'Cape 1969 (IDW)',
 'Capt Action Cat (DE)',
 'Capt Victory & Galactic Range (DE)',
 'Captain America (Marvel)',
 'Captain America 1940s Newspap (Marvel)',
 'Captain America Americas Aven (Marvel)',
 'Captain America Annual (Marvel)',
 'Captain America Black Panthe (Marvel)',
 'Captain America Comics (Marvel)',
 'Captain America Corps (Marvel)',
 'Captain America Fighting Aven (Marvel)',
 'Captain America First Vengean (Marvel)',
 'Captain America Forever Allie (Marvel)',
 'Captain America Hail Hydra (Marvel)',
 'Captain America Korvac Saga (Marvel)',
 'Captain America Living Legen (Marvel)',
 'Captain America Man Out of Ti (Marvel)',
 'Captain America Mgc (Marvel)',
 'Captain America Patriot (Marvel)',
 'Captain America Reborn (Marvel)',
 'Captain America Reborn Mgc (Marvel)',
 'Captain America Road To War (Marvel)',
 'Captain America Sam Wilson (Marvel)',
 'Captain America Steve Rogers (Marvel)',
 'Captain America 

**Saturday Night**
Finally finished attempting all titles for scraping. Need a list of titles that weren't scraped yet to make the look up more efficient.

In [9]:
#  Count records.
query = """
    SELECT * from comic_trans;
"""

In [10]:
# Execute the query
cur.execute(query)

In [11]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [12]:
temp_df.head(3)

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG)
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG)
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG)


In [13]:
temp_df['title'] = ( temp_df['title_and_num'].apply(dfc.cut_issue_num) )

In [14]:
temp_df.head()

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title,title
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG),Filler Bunny
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG),Gargoyles
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG),Royal Historian of Oz
3,5,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-14 19:49:40,00399,Royal Historian of Oz (SLG),Royal Historian of Oz
4,6,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-19 10:39:04,00237,Royal Historian of Oz (SLG),Royal Historian of Oz


In [15]:
temp_df['title'] = (temp_df['title'].apply(lambda x : x.replace('&' ,'and'))
                    .apply(lambda x : x.replace('?' ,''))
                    .apply(lambda x : x.replace('/' ,' '))
                   )

In [16]:
titles = list(temp_df['title'].unique())

In [17]:
sample_titles = titles[:300]

In [18]:
sample_titles

['Filler Bunny',
 'Gargoyles',
 'Royal Historian of Oz',
 'Warlord of Io and Other Storie',
 'Afterlife With Archie',
 'Afterlife With Archie Magazin',
 'Archie',
 'Boys',
 'Archie and Friends Double Diges',
 'Archie and Friends Double Doubl',
 'Archie 1941',
 'Archie and Me Comics Digest',
 'Archie and Me Jumbo Comics Di',
 'Archie Collectors E',
 'Archie Comics Annual Digest',
 'Archie Comics Double Digest',
 'Archie Double Digest',
 'Archie Jumbo Comics Digest',
 'Archie Meets Batman 66',
 'Archie Meets Ramones One Sho',
 'Archie Vs Sharknado One Shot',
 'Archies',
 'Archies Halloween Spectacula',
 'Archies One Shot Cvr A Jaime',
 'Archies Superteens Vs Crusade',
 'B and V Friends Double Digest',
 'B and V Friends Halloween Annua',
 'B and V Friends Jumbo Comics Di',
 'Betty and Veronica',
 'Betty and Veronica Comics Annua',
 'Betty and Veronica Comics Diges',
 'Betty and Veronica Comics Doubl',
 'Betty and Veronica Friends Fore',
 'Betty and Veronica Holiday Annu',
 'Betty and Vero

Get list, sorted by qty sold

In [19]:
qtys = temp_df.groupby(['title'], as_index=False).qty_sold.sum().sort_values(by=['qty_sold'], ascending=False)

In [20]:
qtys.head()

,title,qty_sold
553,Batman,9562
6699,Walking Dead,6856
229,Amazing Spider-Man,5828
5098,Saga,5542
5928,Superman,5197


In [21]:
done_titles = titles[:300]

In [22]:
titles_needed_df = qtys.loc[~qtys['title'].isin(done_titles)]

In [23]:
titles_needed_df.shape

(6771, 2)

In [24]:
titles_need_list = list(titles_needed_df.title.unique())

In [25]:
#367+246+151
827+151+376+524+5+47+1662+3+162+155+15+295+927+143+60

5352

In [26]:
new_start=5352 #1932

In [27]:
titles_searching = titles_need_list[new_start:]

In [28]:
titles_searching

['Hack Slash Annual 2011 Hatche',
 'Disney Magic Kingdom Comics',
 'Death of Cerebus In Hell',
 'Robocop Beta One Sho',
 'Hellcyon',
 'Bomb Queen All Girl Spec',
 'Iron Man Titanium',
 'True Believers Venom Shiver',
 'Locke and Key Clockworks #1 and',
 'Death of Dracula',
 'Skeptics',
 'Iron Siege',
 'Irredeemable Incorruptible',
 'Batman Doc Savage Special',
 'Bolts',
 'Anthem',
 'Red Sonja Annual',
 'Sif',
 'Super Dinosaur Dlx Coloring B',
 'Die Hard Year One',
 '1 For $1 X',
 'Black Widow and Marvel Girls',
 'Batman Gotham After Midnight',
 'Cavalry Shield 50th Anniversa',
 'Df Nemesis',
 'Richard Starks Parker the Out',
 'Rem 8',
 'Die Kitty Die',
 'Devil Is Due In Dreary',
 'Lcsd 2015 All New All Differe',
 'Mighty Man (One Shot) (Mr',
 'Jim Hensons Labyrinth 2017 Sp',
 'Fall Out Toy Works',
 'Harbinger Wars 2 Prelude',
 'Retailer Summit 2019 Venom',
 'Last Space Race',
 'Disney Muppets Presents Meet',
 'Jingle Belle Gift Wrapped Spe',
 'Black Panther Sound and Fury',
 'Disney Gia

In [29]:
len(titles_searching)

1419

So now we have subset of files we need to recheck.

In [48]:
work.columns

Index(['comic_title', 'title', 'index', 'filename', 'search_path'], dtype='object')

In [62]:
subset = work.loc[work['title'].isin(titles_searching)].copy()

In [63]:
tgt_titles = list(subset['comic_title'])

In [82]:
tgt_titles[1404:]

['X-Men Phoenix Force Handboo (Marvel)',
 'X-Men Second Coming Finch Va (Marvel)',
 'X-O Manowar (2017) (New Arc) (Other)',
 'X-O Manowar (Ongoing) 2nd Pt (Other)',
 'X-O Manowar 25th Ann Special (Other)',
 'Xena (DE)',
 'Yankee (Other)',
 'Yo-Kai Watch (IDW)',
 'You Can Try Again One Shot (M (Other)',
 'Z Nation (DE)',
 'Zen Hard Bounty (Other)',
 'Zmd Zombies of Mass Destructi (Red 5)',
 'Zodiac (Zenescope)',
 'Zoe Dare Vs Disasteroid (Other)',
 'Zombie Tramp Vs Vampblade (Other)',
 'Zombies Vs Robots Annual 201 (IDW)']

Great. Now let's check these titles and be done with it!

In [84]:
conn.rollback()

In [85]:
for title in tgt_titles[1404:]:
    # Find file name
    
    # escape single count
    title_sql = title.replace("'", "''")
    
    try:
        got_filename = work.loc[work['comic_title']==title]['filename'].values[0]
        client.head_object(Bucket='comrx', Key='covers/' + got_filename)
        print('got ' + got_filename)
        
        # Update comics table on AWS
        img_url = work.loc[work['comic_title']==title]['search_path'].values[0]
        print(img_url)
        
        # Build query string
        query_update = ("UPDATE comics " + 
                        "SET img_url = '" + img_url + "' " +
                        "WHERE comic_title = '" + title_sql +
                        "';"
                        )
        print(query_update)
        
        # Execute query
        cur.execute(query_update)
        conn.commit()
           
    except:
        print('No Match, use generic image')
                # Build query string
        query_update = ("UPDATE comics " + 
                        "SET img_url = '" + generic_img_url + "' " +
                        "WHERE comic_title = '" + title_sql +
                        "';"
                        )
        print(query_update)
        
        # Execute query
        cur.execute(query_update)
        conn.commit()


got x-men_phoenix_force_handboo.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/x-men_phoenix_force_handboo.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/x-men_phoenix_force_handboo.jpg' WHERE comic_title = 'X-Men Phoenix Force Handboo (Marvel)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'X-Men Second Coming Finch Va (Marvel)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'X-O Manowar (2017) (New Arc) (Other)';
No Match, use generic image
UPDATE comics SET img_url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg' WHERE comic_title = 'X-O Manowar (Ongoing) 2nd Pt (Other)';
got x-o_manowar_25th_ann_special.jpg
https://comrx.s3-us-west-2.amazonaws.com/covers/x-o_manowar_25th_ann_special.jpg
UPDATE comics SET img_url = 'https://comrx.s3-us-wes

In [61]:
len(tgt_titles)

7202

## Manual URL updater

In [4]:
# Instantiate cursor
cur = conn.cursor()

In [ ]:
#  Count records.
query = """
    SELECT * from comics LIMIT 100;
"""

#  Count records.
query_alter = """
    ALTER TABLE comics 
    ADD COLUMN img_url VARCHAR;
"""

tables = """
SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='public'
   AND table_type='BASE TABLE';
   """

ALTER TABLE table_name
ADD COLUMN new_column_name data_type;

# Execute the query
cur.execute(query_alter)

# Execute the query
cur.execute(query)

conn.commit()

# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [25]:
def update_img_url(comic_title, new_url, conn):
    """
    Given comic title, update the url to the image.
    Uses psycopg2 connection
    """
    # escape single count
    title_sql = comic_title.replace("'", "''")
    
    # Open cursor
    cur = conn.cursor()
    
    # Form query
    query = ("UPDATE comics " + 
             "SET img_url = '" + new_url + "' " +
             "WHERE comic_title = '" + title_sql +
             "';")
    # Execute
    cur.execute(query)
    conn.commit()
    
    # Check
    query = ("SELECT * FROM comics " + 
                "WHERE comic_title = '" + title_sql +
             "';")
    # Execute
    cur.execute(query)
    conn.commit()
    
    # Check results
    temp_df = pd.DataFrame(cur.fetchall())
    temp_df.columns = [col.name for col in cur.description]
    
    return temp_df
    

In [49]:
url = 'https://comrx.s3-us-west-2.amazonaws.com/covers/avengers_(2010).jpg'

In [50]:
d = update_img_url('Avengers (2010) (Marvel)', url, conn)

In [51]:
d

,comic_id,comic_title,img_url
0,433,Avengers (2010) (Marvel),https://comrx.s3-us-west-2.amazonaws.com/covers/avengers_(2010).jpg


In [16]:
q = """select * from comics where comic_title like '%Wicked%';"""

In [21]:

cur.execute(q)

In [22]:
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]
temp_df

,comic_id,comic_title,img_url
0,6930,Wicked & Divine 1373 Cvr B Ke (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
1,6937,Wicked & Divine (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
2,6932,Wicked & Divine 1923 Cvr A Mc (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
3,6934,Wicked & Divine 455 Ad (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
4,6936,Wicked & Divine Funnies (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
5,3265,Image Firsts Wicked & Divine (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
6,6929,Wicked & Divine 1373 Cvr A Mc (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
7,6931,Wicked & Divine 1831 One Sho (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
8,6935,Wicked & Divine Christmas Ann (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
9,6933,Wicked & Divine 1923 Cvr B Ko (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
